In [1]:
import pandas as pd

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

# Load model predictions and labels
val_predictions_model1 = pd.read_csv('val_predictions_roberta.csv')['Prediction']
val_predictions_model2 = pd.read_csv('val_predictions_bert.csv')['Prediction']
val_predictions_model3 = pd.read_csv('val_predictions_xlnet.csv')['Prediction']
val_labels = pd.read_csv('val_predictions_roberta.csv')['Label']

test_predictions_model1 = pd.read_csv('test_predictions_roberta.csv')['Prediction']
test_predictions_model2 = pd.read_csv('test_predictions_bert.csv')['Prediction']
test_predictions_model3 = pd.read_csv('test_predictions_xlnet.csv')['Prediction']
test_labels = pd.read_csv('test_predictions_roberta.csv')['Label']



In [3]:
print(test_predictions_model1.shape)
print(test_predictions_model2.shape)
print(test_predictions_model3.shape)


(909,)
(909,)
(909,)


In [ ]:
print(test_labels.shape)

(909,)


In [ ]:
def weighted_average(predictions, weights):
    weighted_preds = np.zeros(predictions[0].shape)
    for pred, weight in zip(predictions, weights):
        weighted_preds += pred * weight
    return weighted_preds / np.sum(weights)


In [ ]:


# Function to evaluate the ensemble using RMSE (or choose your metric)
def evaluate_ensemble(weights):
    ensemble_val_predictions = weighted_average(
        [val_predictions_model1, val_predictions_model2, val_predictions_model3], weights)
    return np.sqrt(mean_squared_error(val_labels, ensemble_val_predictions))

# Grid search to find the best weights
best_rmse = float('inf')
best_weights = None
for w1 in np.arange(0, 1.1, 0.1):
    for w2 in np.arange(0, 1.1 - w1, 0.1):
        w3 = 1 - w1 - w2
        rmse = evaluate_ensemble([w1, w2, w3])
        if rmse < best_rmse:
            best_rmse = rmse
            best_weights = [w1, w2, w3]

print("Best Weights:", best_weights)
print("Best RMSE on Validation Data:", best_rmse)

# Apply best weights to test data
ensemble_test_predictions = weighted_average(
    [test_predictions_model1, test_predictions_model2, test_predictions_model3], best_weights)

# Evaluate ensemble on test data
ensemble_test_rmse = np.sqrt(mean_squared_error(test_labels, ensemble_test_predictions))
print("RMSE on Test Data:", ensemble_test_rmse)


Best Weights: [1.0, 0.0, 0.0]
Best RMSE on Validation Data: 1.1584588845600858
RMSE on Test Data: 1.113076226091206


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [7]:
# Combine predictions
X_train = pd.DataFrame({'Model1': val_predictions_model1, 'Model2': val_predictions_model2, 'Model3': val_predictions_model3})
y_train =  pd.read_csv('val_predictions_roberta.csv')['Label']
X_test = pd.DataFrame({'Model1': test_predictions_model1, 'Model2': test_predictions_model2, 'Model3': test_predictions_model3})
y_test = pd.read_csv('test_predictions_roberta.csv')['Label']

# Split data (optional)
#X_train, X_val, y_train, y_val = train_test_split(combined_preds, labels, test_size=0.2, random_state=42)

# Train meta-model
meta_model = LinearRegression()
meta_model.fit(X_train, y_train)

# # Evaluate meta-model
# predictions_val = meta_model.predict(X_val)
# print("Validation MSE:", mean_squared_error(y_val, predictions_val))

# Apply meta-model to test data
# Load test predictions in similar fashion
# test_model1_preds = pd.read_csv('test_model1_predictions.csv')['Prediction']
# test_model2_preds = pd.read_csv('test_model2_predictions.csv')['Prediction']
# test_model3_preds = pd.read_csv('test_model3_predictions.csv')['Prediction']

#test_combined_preds = pd.DataFrame({'Model1': test_model1_preds, 'Model2': test_model2_preds, 'Model3': test_model3_preds})
final_predictions = meta_model.predict(X_test)


In [8]:
ensemble_test_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
print("RMSE on Test Data:", ensemble_test_rmse)

RMSE on Test Data: 1.0930427505033127


In [11]:
from sklearn import metrics
from scipy.spatial import distance
from scipy import stats
# Calculate Pearson correlation and cosine similarity
pearson_corr = stats.pearsonr(final_predictions, y_test)[0]
cosine_sim = 1 - distance.cosine(final_predictions, y_test)
print(f"Pearson Correlation: {pearson_corr}")
print(f"Cosine Similarity: {cosine_sim}")

Pearson Correlation: 0.8514515062935436
Cosine Similarity: 0.9829382409265901


In [ ]:
pip install xgboost


In [ ]:
import xgboost as xgb
meta_model_xgb = xgb.XGBRegressor()
meta_model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
final_predictions_xgb = meta_model_xgb.predict(X_test)

In [ ]:
ensemble_test_rmse_xgb = np.sqrt(mean_squared_error(y_test, final_predictions_xgb))
print("RMSE on Test Data:", ensemble_test_rmse_xgb)

RMSE on Test Data: 1.2628693903572086


In [ ]:
from sklearn.ensemble import RandomForestRegressor
meta_model_rf = RandomForestRegressor()
meta_model_rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
final_predictions_rf = meta_model_rf.predict(X_test)

In [ ]:
ensemble_test_rmse_rf = np.sqrt(mean_squared_error(y_test, final_predictions_rf))
print("RMSE on Test Data:", ensemble_test_rmse_rf)

RMSE on Test Data: 1.1808208813344128


In [5]:
pip install scikit-optimize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
# Bayesian optimization

In [12]:
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from skopt.space import Real, Categorical, Integer

# Define the search space for hyperparameters
search_space = {
    'n_estimators': Integer(100, 500),
    'max_depth': Integer(3, 30),
    'min_samples_split': Real(0.01, 0.1),
    'min_samples_leaf': Integer(1, 4),
    'max_features': Categorical(['sqrt', 'log2', None]),
}

# Initialize the Bayesian optimizer with the RandomForestRegressor and the search space
bayes_search = BayesSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    search_spaces=search_space,
    n_iter=32,  # Number of iterations
    scoring='neg_mean_squared_error',  # Or your desired scoring
    cv=3,  # Cross-validation splitting strategy
    n_jobs=-1,  # Number of jobs to run in parallel
    refit=True,
    random_state=42
)

# Assuming stacked_val_predictions and val_labels are your features and labels respectively
bayes_search.fit(X_train, y_train)

# Best parameters found for RandomForestRegressor
print("Best parameters for RandomForest:", bayes_search.best_params_)

best_rmse = np.sqrt(-bayes_search.best_score_)
print("Best RMSE on Validation Data:", best_rmse)

# For Linear Regression, it's uncommon to perform Bayesian optimization because it typically has fewer hyperparameters to tune.
# However, if you want to optimize hyperparameters like 'fit_intercept' or 'normalize', you can define a search space for those.


/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/usr/local/lib/python3.10/dist-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Best parameters for RandomForest: OrderedDict([('max_depth', 3), ('max_features', None), ('min_samples_leaf', 4), ('min_samples_split', 0.0919122704037716), ('n_estimators', 500)])
Best RMSE on Validation Data: 1.1755543885289377


In [13]:
final_predictions_rf_cv = bayes_search.predict(X_test)

In [14]:
ensemble_test_rmse_rf_cv = np.sqrt(mean_squared_error(y_test, final_predictions_rf_cv))
print("RMSE on Test Data:", ensemble_test_rmse_rf_cv)

RMSE on Test Data: 1.0879793465143763


In [15]:
# Calculate Pearson correlation and cosine similarity
pearson_corr = stats.pearsonr(final_predictions_rf_cv, y_test)[0]
cosine_sim = 1 - distance.cosine(final_predictions_rf_cv, y_test)
print(f"Pearson Correlation: {pearson_corr}")
print(f"Cosine Similarity: {cosine_sim}")

Pearson Correlation: 0.853139583236503
Cosine Similarity: 0.9831186668356847
